In [ ]:
import pandas as pd


Importation des portefeuilles à une date T
Importation des prix de clôture des titres de l'univers GIP

In [ ]:
ALL_Portfolio = pd.read_excel(r'C:\Sharepoint GIP\Gestion Indépendante de patrimoines GIP SA\Docs - Documents\gip sa\Scripts\Risk\Data_Raw\ALL_PORTEFEUILLE_GIP_ANONYME.xlsx')

#Renommer la colonne code portefeuille

ALL_Portfolio = ALL_Portfolio.rename(columns={'Code/Numéro/Contrat': 'Code'})

#IMport liste de titres
#Import closing price

Titres = pd.read_excel(r'C:\Sharepoint GIP\Gestion Indépendante de patrimoines GIP SA\Docs - Documents\datas\Liste Titres CRM\ListeTitres (Pour importation CRM).xlsx')
ALL_CLOSING_Price = pd.read_excel(r'C:\Sharepoint GIP\Gestion Indépendante de patrimoines GIP SA\Docs - Documents\gip sa\Scripts\Risk\Data_Raw\ALL_CLOSE_PRICE_GIP.xlsx')


Historique_Vol = 20  #remplir dropdown menu datalore

Sélection du portefeuille et calcul des pondération de chaque actif

In [ ]:
portfolio_code = 150501

Selected_Portfolio = ALL_Portfolio[
    ALL_Portfolio['Code portefeuille'] == portfolio_code
    ].copy()

 # Calculer la valeur totale du marché du portefeuille sélectionné
total_selected_portfolio = Selected_Portfolio['Valeur marché en  (mon. réf.)'].sum()

# Ajouter la colonne 'Pondération'
Selected_Portfolio['Pondération'] = (
        Selected_Portfolio['Valeur marché en  (mon. réf.)'] / total_selected_portfolio
).round(2)

Selected_Portfolio

Création d'un dataframe avec les prix historiques des actifs du portefeuille

Collecte des ISIN du portefeuille

In [ ]:
ISIN_Selected_Portfolio = Selected_Portfolio['Code'].unique()

ISIN_Selected_Portfolio

Filter le datadframe de l'univers titres

In [ ]:
Historical_Price_Selected_Portfolio = ALL_CLOSING_Price[ALL_CLOSING_Price['Code'].isin(ISIN_Selected_Portfolio)]

Historical_Price_Selected_Portfolio = Historical_Price_Selected_Portfolio.drop(columns =['Pr.','Libellé','Abréviation','Monnaie','Genre']).T

# Faire de la ligne "Code" les noms des colonnes
Historical_Price_Selected_Portfolio.columns = Historical_Price_Selected_Portfolio.iloc[0]

Historical_Price_Selected_Portfolio = Historical_Price_Selected_Portfolio[1:]  # Supprimer la ligne devenue les colonnes

Historical_Price_Selected_Portfolio





In [ ]:
Historical_Price_Selected_Portfolio = Historical_Price_Selected_Portfolio.ffill()

Historical_Price_Selected_Portfolio = Historical_Price_Selected_Portfolio.apply(pd.to_numeric, errors='coerce')

Historical_Price_Selected_Portfolio_percent = Historical_Price_Selected_Portfolio.pct_change(fill_method=None)

Historical_Price_Selected_Portfolio_percent = Historical_Price_Selected_Portfolio_percent.dropna()

Historical_Price_Selected_Portfolio_percent



Estimation Mean Variance Portfolio

In [ ]:

import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Historical_Price_Selected_Portfolio_percent)

# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

Dessiner la composition du portefeuille

In [ ]:
# Plotting the composition of the portfolio

ax = rp.plot_pie(w=w, title='Sharpe Mean Variance', others=0.05, nrow=25, cmap = "tab20",
                 height=6, width=10, ax=None)

Calculer la frontière efficiente

In [ ]:
points = 50 # Number of points of the frontier

frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

display(frontier.T.head())

Dessiner la frontière efficiente

In [ ]:
# Plotting the efficient frontier

label = 'Max Risk Adjusted Return Portfolio' # Title of point
mu = port.mu # Expected returns
cov = port.cov # Covariance matrix
returns = port.returns # Returns of the assets

ax = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm,
                      rf=rf, alpha=0.05, cmap='viridis', w=w, label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)